<a href="https://colab.research.google.com/github/AmirEfi/Prompt-Engineering-of-LLMs-in-Defect-Detection/blob/master/Defect_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

ds = load_dataset("ZHENGRAN/code_ujb_defectdetection")

In [ ]:
df = ds['train'].to_pandas()

In [ ]:
df.head()

,bug_id,task_id,function_signature,prompt_chat,code,defective,project,prompt_complete
0,49,000bd8114192f589e849a9f2a68d4edaaec806bd8ed37b...,public Object generateId(Object forPojo),"I want you to act as a code defect detector, w...",public Object generateId(Object forPojo) {...,True,JacksonDatabind,/**\n * Perform a binary search on a sorted ar...
1,56,005766411469575f53bb173eea8c7bf90dde1a7f0a41ba...,public int[] getCounts(int index),"I want you to act as a code defect detector, w...",public int[] getCounts(int index) {\n ...,False,Math,/**\n * Perform a binary search on a sorted ar...
2,9,009c707abb74fee8886efd9438bf903d296c8a30fda3f2...,public static byte[] encodeBase64(byte[] binar...,"I want you to act as a code defect detector, w...",public static byte[] encodeBase64(byte[] b...,False,Codec,/**\n * Perform a binary search on a sorted ar...
3,44,00d45f48dfdcff13baeaca61654b0467fae727dfc323d7...,public ChecksumCalculatingInputStream(final Ch...,"I want you to act as a code defect detector, w...",public ChecksumCalculatingInputStream(fina...,True,Compress,/**\n * Perform a binary search on a sorted ar...
4,64,0158d98cfe414fb8a3a32ae6d96b1f61a405dfb4990b38...,@Override\n protected VectorialPointValuePa...,"I want you to act as a code defect detector, w...",@Override\n protected VectorialPointVal...,True,Math,/**\n * Perform a binary search on a sorted ar...


In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

In [ ]:
prompts = df['prompt_chat']

In [ ]:
output_model = []
for pr in prompts:
  pr = pr.replace("Please", "ONLY")
  prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

  USER: {pr}

  ASSISTANT:
  '''
  if len(prompt_template) > 1000:
    output_model.append("False")
    continue
  response=lcpp_llm(prompt=prompt_template, max_tokens=150, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=False)
  if "Yes" in response["choices"][0]["text"]:
    output_model.append("True")
  else:
    output_model.append("False")

In [ ]:
print(output_model)

['True', 'False', 'False', 'False', 'False', 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'True', 'False', 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'True', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'True', 'False', 'False', 'True', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'F

In [ ]:
df['Output Model'] = [value == 'True' for value in output_model]

In [ ]:
df.head()

,bug_id,task_id,function_signature,prompt_chat,code,defective,project,prompt_complete,Output Model
0,49,000bd8114192f589e849a9f2a68d4edaaec806bd8ed37b...,public Object generateId(Object forPojo),"I want you to act as a code defect detector, w...",public Object generateId(Object forPojo) {...,True,JacksonDatabind,/**\n * Perform a binary search on a sorted ar...,True
1,56,005766411469575f53bb173eea8c7bf90dde1a7f0a41ba...,public int[] getCounts(int index),"I want you to act as a code defect detector, w...",public int[] getCounts(int index) {\n ...,False,Math,/**\n * Perform a binary search on a sorted ar...,False
2,9,009c707abb74fee8886efd9438bf903d296c8a30fda3f2...,public static byte[] encodeBase64(byte[] binar...,"I want you to act as a code defect detector, w...",public static byte[] encodeBase64(byte[] b...,False,Codec,/**\n * Perform a binary search on a sorted ar...,False
3,44,00d45f48dfdcff13baeaca61654b0467fae727dfc323d7...,public ChecksumCalculatingInputStream(final Ch...,"I want you to act as a code defect detector, w...",public ChecksumCalculatingInputStream(fina...,True,Compress,/**\n * Perform a binary search on a sorted ar...,False
4,64,0158d98cfe414fb8a3a32ae6d96b1f61a405dfb4990b38...,@Override\n protected VectorialPointValuePa...,"I want you to act as a code defect detector, w...",@Override\n protected VectorialPointVal...,True,Math,/**\n * Perform a binary search on a sorted ar...,False


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df['Output Model'], df['defective'])
print(f'Accuracy: {accuracy * 100:.2f} %')

Accuracy: 51.49 %
